In [1]:
import cobra
import cobra.test
import pickle
import pandas as pd
from multime.auxotroph_analysis import load_model

/home/sbrg-cjlloyd/cobrapy/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


In [2]:
me = load_model.load_me_model()

12.0 protein_b3946 --> EG11905-MONOMER
10000.0
coa_p not in model
10000.0
thf_p not in model
10000.0
nadp_p not in model
10000.0
nad_p not in model
10000.0
ribflv_p not in model
10000.0
10000.0
q8_p not in model
10000.0
2dmmq8_p not in model
10000.0
mqn8_p not in model
10000.0
fmn_p not in model
10000.0
fad_p not in model


In [3]:
ijo = cobra.io.load_json_model('/home/sbrg-cjlloyd/Desktop/ecoli_M_models/iJO1366.json')
vit_to_rs = {}
# Get reactions catalyzed by proteins that use these metabolites as prosthetic groups
for mets in [['thmpp'], ['btn'], ['pydx5p'], ['cbl1', 'adocbl'], 
             ['ribflv', 'fad', 'fmn', 'rbflvrd', 'fadh2', 'fmnh2'],
            ['nad', 'nadp', 'nadh', 'nadph']]:
    rs = []
    for met in mets:
        met = met+'_c'
        for c in me.metabolites.get_by_id(met).reactions:
            if not c.id.startswith('formation_') or c.id.endswith('alt'):
                continue
            for r in me.metabolites.get_by_id(c.id.replace('formation_', '')).reactions:
                try:
                    rs.append(r.stoichiometric_data.id)
                except:
                    continue
    vit_to_rs[', '.join(mets)] = ', '.join(set(rs))
    
# Get reactions that use these metabolites as coenzymes
for mets in [['nad', 'nadp', 'nadh', 'nadph'], ['thf', '10fthf', '5fthf', '5mthf', 'methf', 'mlthf', 'mththf'],
             ['coa'], ['ribflv', 'fad', 'fmn', 'rbflvrd', 'fadh2', 'fmnh2']]:
    rs = []
    for met in mets:
        for comp in ['_c', '_p', '_e']:
            if met+comp not in ijo.metabolites:
                continue
            for r in ijo.metabolites.get_by_id(met + comp).reactions:
                if 'biomass' in r.id.lower():
                    continue
                rs.append(r.id)
    key = ', '.join(mets)
    if key in vit_to_rs:
        vit_to_rs[key] += ', '
    else:
        vit_to_rs[key] = ''
    vit_to_rs[key] += ', '.join(set(rs))
    
# loop 1 and 2 can add the same reaction twice. account for that
for key in list(vit_to_rs.keys()):
    print(key)
    print(len(vit_to_rs[key].split(', ')))
    vit_to_rs[key] = ', '.join(set(vit_to_rs[key].split(', ')))
    print(len(vit_to_rs[key].split(', ')))

thmpp
12
12
btn
1
1
pydx5p
62
62
cbl1, adocbl
3
3
ribflv, fad, fmn, rbflvrd, fadh2, fmnh2
111
107
nad, nadp, nadh, nadph
227
225
thf, 10fthf, 5fthf, 5mthf, methf, mlthf, mththf
19
19
coa
71
71


In [4]:
pd.DataFrame.from_dict(vit_to_rs, orient='index').to_csv('/home/sbrg-cjlloyd/Dropbox/vitamin_to_blocked_reactions.csv')

In [4]:
pd.DataFrame.from_dict(vit_to_rs, orient='index').rename(lambda x: x.split(',')[0]).rename(load_model.met_to_name)

,0
Thiamine \n diphosphate,"POR51, SEPHCHCS, GLXCL, DXPS, TKT2, ACHBS, AKG..."
Biotin,ACCOAC
Pyridoxal \n 5'-phosphate,"LEUTAi, S_transfer_to_IscU_mod_1:2Fe1S, ICYSDS..."
Cobalamin,"MMM, METS, ETHAAL"
Riboflavin,"ACOAD4f, ARBTNR1, G3PD5, FRD2, FLDR22, G3PD6, ..."
NAD,"3OAR100, DHQS, 3OAR160, TDPDRR, 3OAR120, SSALx..."
Tetrahydrofolate,"MTHTHFSs, METS, THFAT, FTHFD, ULA4NFT, FOMETRi..."
Coenzyme A,"FACOAL80t2pp, MCITS, CRNDCAL2, FACOAE160, ACAC..."


In [7]:
df = pd.read_csv('vitamin_to_blocked_reactions.csv', index_col=0)

In [11]:
from optaux.helper_functions.characterize_auxotrophs import get_blocked_biomass
vit_to_blocked_mets = pd.DataFrame()
for i in df.index:
    print(i)
    model = ijo.copy()
    for r in df.loc[i, '0'].split(', '):
        if r in model.reactions:
            model.reactions.get_by_id(r).knock_out()
        elif r[:-1] in model.reactions:
            model.reactions.get_by_id(r[:-1]).knock_out()
        elif r == 'BTS6':
            model.reactions.get_by_id('BTS5').knock_out()
        else:
            print(r)
    vit_to_blocked_mets.loc[i, 'Metabolites'] = ', '.join(get_blocked_biomass(model, []))

thmpp
btn
pydx5p
S_transfer_to_SufBCD_mod_2:fe2
S_transfer_to_SufBCD_mod_1:2Fe1S
S_transfer_to_SufBCD_mod_1:2Fe2S_mod_2:fe2
S_transfer_to_IscU_mod_1:2Fe2S_mod_1:2Fe1S
S_transfer_to_IscU_mod_1:2Fe2S_mod_2:fe2
S_transfer_to_SufBCD_mod_1:2Fe2S_mod_1:2Fe1S
S_transfer_to_IscU_mod_2:fe2
S_transfer_to_IscU_mod_1:2Fe1S
cbl1, adocbl
ribflv, fad, fmn, rbflvrd, fadh2, fmnh2
L__LACD3
CU2R
L__LACD2
nad, nadp, nadh, nadph
thf, 10fthf, 5fthf, 5mthf, methf, mlthf, mththf
coa


In [18]:
vit_to_blocked_mets.to_csv('/home/sbrg-cjlloyd/Dropbox//vit_to_blocked_mets.csv')

In [5]:
df = pd.DataFrame.from_dict(load_model.currency_met_to_synthesis_rxn, orient='index')

In [17]:
cofactor_to_rxn = df[df>''].dropna(axis=0).rename(load_model.met_to_name).rename(lambda x: x.replace('\n', ''))

for i in cofactor_to_rxn.index:
    cofactor_to_rxn.loc[i, 'Reaction Name'] = ijo.reactions.get_by_id(cofactor_to_rxn.loc[i, 0]).name
cofactor_to_rxn.rename({0: 'Reaction ID'}, axis=1).to_csv('/home/sbrg-cjlloyd/Dropbox/cofactor_to_rxn.csv')

In [24]:
color_dict = {'Nitrogen': 'green', 'Carbon': 'black', 'Phosphorus':'red', 'Sulfur':'yellow',
              'Anaerobic Nitrogen': '#90ee90', 'Anaerobic Carbon': '#D3D3D3',
              'Anaerobic Phosphorus': '#ff6961', 'Anaerobic Sulfur': '#ffffe0'}

In [33]:
color_df = pd.DataFrame.from_dict(color_dict, orient='index').style.applymap(lambda x: 'background-color: %s' % x)

In [37]:
color_df.to_excel('/home/sbrg-cjlloyd/Dropbox//color_df.xlsx')